In [ ]:
import json
import websocket
import datetime
import numpy as np
import pandas as pd
from datetime import datetime, timezone
import time

In [ ]:
def my_json(json_message):
    size = len(np.array(json_message['bids'])[:,0])    
    return {
        **{'ts':[time.mktime(datetime.now().timetuple())]},
        **{'bp'+str(key):[float(value)] for key,value in zip(np.arange(0,size)+1,np.array(json_message['bids'])[:,0])},
        **{'bs'+str(key):[float(value)] for key,value in zip(np.arange(0,size)+1,np.array(json_message['bids'])[:,1])},
        **{'ap'+str(key):[float(value)] for key,value in zip(np.arange(0,size)+1,np.array(json_message['asks'])[:,0])},
        **{'as'+str(key):[float(value)] for key,value in zip(np.arange(0,size)+1,np.array(json_message['asks'])[:,1])}}
    
def my_json_0(size):    
    return {
        **{'ts':[time.mktime(datetime.now().timetuple())]},
        **{'bp'+str(key):[value] for key,value in zip(np.arange(0,size)+1,(np.arange(0,size)+1)*np.nan)},
        **{'bs'+str(key):[value] for key,value in zip(np.arange(0,size)+1,(np.arange(0,size)+1)*np.nan)},
        **{'ap'+str(key):[value] for key,value in zip(np.arange(0,size)+1,(np.arange(0,size)+1)*np.nan)},
        **{'as'+str(key):[value] for key,value in zip(np.arange(0,size)+1,(np.arange(0,size)+1)*np.nan)}}
        

        

In [ ]:
def my_json(json_message):
    size = len(np.array(json_message['bids'])[:,0])    
    return {
        **{'primary_key':[datetime.now()]},
        **{'bp'+str(key):[float(value)] for key,value in zip(np.arange(0,size)+1,np.array(json_message['bids'])[:,0])},
        **{'bs'+str(key):[float(value)] for key,value in zip(np.arange(0,size)+1,np.array(json_message['bids'])[:,1])},
        **{'ap'+str(key):[float(value)] for key,value in zip(np.arange(0,size)+1,np.array(json_message['asks'])[:,0])},
        **{'as'+str(key):[float(value)] for key,value in zip(np.arange(0,size)+1,np.array(json_message['asks'])[:,1])}}
    
def my_json_0(size):    
    return {
        **{'primary_key':[datetime.now()]},
        **{'bp'+str(key):[value] for key,value in zip(np.arange(0,size)+1,(np.arange(0,size)+1)*np.nan)},
        **{'bs'+str(key):[value] for key,value in zip(np.arange(0,size)+1,(np.arange(0,size)+1)*np.nan)},
        **{'ap'+str(key):[value] for key,value in zip(np.arange(0,size)+1,(np.arange(0,size)+1)*np.nan)},
        **{'as'+str(key):[value] for key,value in zip(np.arange(0,size)+1,(np.arange(0,size)+1)*np.nan)}}

In [ ]:

df_0 = pd.DataFrame.from_dict(my_json_0(5))

def ws_trades(df_0): 
    symbol = 'BTCUSDT'
    depth = 5
    socket = 'wss://stream.binance.com:9443/ws/{}@depth{}'.format(symbol.lower(),
                                                                depth)
    
    
    def on_message(wsapp,message):  
        json_message = json.loads(message)
        handle_trades(json_message)

    def on_error(wsapp,error):
        print(error)

    wsapp = websocket.WebSocketApp(socket, on_message=on_message, on_error=on_error)
    wsapp.run_forever()
    
def handle_trades(json_message):    
    df = pd.DataFrame.from_dict(my_json(json_message))
    df_0 = df_0.append(df, ignore_index = True)
    
    print(df_0)
    
    
    
    print("-----------------------")

In [67]:
class StreamingData():
    def __init__(self):
        self.depth = 5     
        self.df_0 = None           
        self.symbol = 'BTCUSDT'        
        self.socket = 'wss://stream.binance.com:9443/ws/{}@depth{}'.format(self.symbol.lower(),self.depth)
        
    def handle_trades(self,json_message):    
        df = pd.DataFrame.from_dict(self.my_json(json_message))
        self.df_0 = pd.concat([self.df_0, df], axis=0)
        print(self.my_return())
        
    def my_return(self):
        return self.preprocessing_streamed_data(self.df_0, rolling_window=2)

    def on_message(self,wsapp,message):  
        json_message = json.loads(message)
        self.handle_trades(json_message)

    def on_error(self,wsapp,error):
        print(error)

    def start(self):        
        self.df_0 = pd.DataFrame.from_dict(self.my_json_0(self.depth))
        print(self.df_0)
        wsapp = websocket.WebSocketApp(self.socket, on_message=self.on_message, on_error=self.on_error)
        wsapp.run_forever()
        
    def preprocessing_streamed_data(self,df_ob, rolling_window=2):
        '''preprocessing of data for streamed data'''                
        
        # aggregating by seconds     
        df_agg = df_ob.groupby(pd.Grouper(key='primary_key', axis=0, freq='S')).mean()
        # applying rolling window of rolling_window lenght
        df_agg = df_agg.rolling(str(rolling_window)+'S').mean()
        # moving the index as a column
        df_agg.reset_index(inplace=True)

        return df_agg
        
    def my_json(self, json_message):
        size = len(np.array(json_message['bids'])[:,0])    
        return {
            **{'primary_key':[datetime.now()]},
            **{'bp'+str(key):[float(value)] for key,value in zip(np.arange(0,size)+1,np.array(json_message['bids'])[:,0])},
            **{'bs'+str(key):[float(value)] for key,value in zip(np.arange(0,size)+1,np.array(json_message['bids'])[:,1])},
            **{'ap'+str(key):[float(value)] for key,value in zip(np.arange(0,size)+1,np.array(json_message['asks'])[:,0])},
            **{'as'+str(key):[float(value)] for key,value in zip(np.arange(0,size)+1,np.array(json_message['asks'])[:,1])}}
    
    def my_json_0(self, size):    
        return {
            **{'primary_key':[datetime.now()]},
            **{'bp'+str(key):[value] for key,value in zip(np.arange(0,size)+1,(np.arange(0,size)+1)*np.nan)},
            **{'bs'+str(key):[value] for key,value in zip(np.arange(0,size)+1,(np.arange(0,size)+1)*np.nan)},
            **{'ap'+str(key):[value] for key,value in zip(np.arange(0,size)+1,(np.arange(0,size)+1)*np.nan)},
            **{'as'+str(key):[value] for key,value in zip(np.arange(0,size)+1,(np.arange(0,size)+1)*np.nan)}}

        

    

In [68]:
w = StreamingData()
w.start()
w.my_return()



                 primary_key  bp1  bp2  bp3  bp4  bp5  bs1  bs2  bs3  bs4  \
0 2022-05-26 13:20:48.987717  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN   

   ...  ap1  ap2  ap3  ap4  ap5  as1  as2  as3  as4  as5  
0  ...  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  

[1 rows x 21 columns]
          primary_key      bp1       bp2      bp3       bp4       bp5  \
0 2022-05-26 13:20:48      NaN       NaN      NaN       NaN       NaN   
1 2022-05-26 13:20:49      NaN       NaN      NaN       NaN       NaN   
2 2022-05-26 13:20:50  29413.4  29413.31  29413.2  29413.17  29412.94   

       bs1      bs2      bs3      bs4  ...       ap1       ap2       ap3  \
0      NaN      NaN      NaN      NaN  ...       NaN       NaN       NaN   
1      NaN      NaN      NaN      NaN  ...       NaN       NaN       NaN   
2  1.53506  0.04125  0.00864  0.01699  ...  29413.41  29413.87  29414.38   

        ap4       ap5      as1      as2      as3      as4   as5  
0       NaN       NaN      NaN      NaN 

,primary_key,bp1,bp2,bp3,bp4,bp5,bs1,bs2,bs3,bs4,...,ap1,ap2,ap3,ap4,ap5,as1,as2,as3,as4,as5
0,2022-05-26 13:20:48,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2022-05-26 13:20:49,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2022-05-26 13:20:50,29413.4,29413.31,29413.20,29413.1700,29412.9400,1.535060,0.04125,0.00864,0.016990,...,29413.41,29413.87,29414.380,29414.72,29414.79,2.187220,0.112320,0.15503,0.101820,0.05000
3,2022-05-26 13:20:51,29413.4,29413.31,29413.20,29413.1700,29412.9400,1.535060,0.04125,0.00864,0.016990,...,29413.41,29413.87,29414.380,29414.72,29414.79,2.187220,0.112320,0.15503,0.101820,0.05000
4,2022-05-26 13:20:52,29413.4,29413.38,29413.31,29413.2000,29413.1700,1.842890,0.05442,0.04125,0.008640,...,29413.41,29413.87,29414.150,29414.38,29414.72,1.402800,0.112320,0.17017,0.155030,0.10182
5,2022-05-26 13:20:53,29413.4,29413.38,29413.31,29413.2125,29413.1825,1.608382,0.05442,0.04125,0.020585,...,29413.41,29413.87,29414.150,29414.38,29414.72,1.833760,0.112320,0.17017,0.155030,0.10182
6,2022-05-26 13:20:54,29413.4,29413.38,29413.31,29413.2625,29413.1975,1.681833,0.05849,0.07102,0.024845,...,29413.41,29414.01,29414.265,29414.55,29414.77,2.250075,0.141245,0.16260,0.128425,0.25491


In [ ]:
ws_trades(df_0)

In [ ]:
  
# initialising dictionaries
ini_dictionary1 = {'nikhil': 1, 'akash' : 5,
                     'manjeet' : 10, 'akshat' : 15}
ini_dictionary2 = {'akash' : 7, 'akshat' : 5,
                                          'm' : 15}

ini_dictionary1.update(ini_dictionary2)
ini_dictionary1  


In [ ]:
json_message ={'lastUpdateId': 19493390097, 
               'bids': [['29724.70000000', '8.90412000'], ['29747.650000', '0.00429000'], ['29724.62000000', '0.08293000'], ['29724.61000000', '0.01998000'], ['29724.57000000', '0.16835000']], 
               'asks': [['29724.71000000', '0.00001000'], ['29724.72000000', '0.00075000'], ['29724.73000000', '0.01237000'], ['29724.75000000', '0.01237000'], ['29724.77000000', '0.01000000']]}


{'bp'+str(key):value for key,value in zip(np.arange(0,5),np.array(json_message['bids'])[:,0]) }
{'bs'+str(key):value for key,value in zip(np.arange(0,5),np.array(json_message['bids'])[:,1]) }
{'bp'+str(key):value for key,value in zip(np.arange(0,5),np.array(json_message['bids'])[:,0]) }
{'bp'+str(key):value for key,value in zip(np.arange(0,5),np.array(json_message['bids'])[:,0]) }


In [ ]:
json_message ={'lastUpdateId': 19493390097, 
               'bids': [['29724.70000000', '8.90412000'], ['nan', '0.00429000'], ['29724.62000000', '0.08293000'], ['29724.61000000', '0.01998000'], ['29724.57000000', '0.16835000']], 
               'asks': [['29724.71000000', '0.00001000'], ['29724.72000000', '0.00075000'], ['29724.73000000', '0.01237000'], ['29724.75000000', '0.01237000'], ['29724.77000000', '0.01000000']]}
aaa(bp1) bid price 1 = 29724.70000000
(bs1) bid size 1 = 8.90412000

(ap1) asd price 1 = 29724.71000000
(as1) ask size 1 = 0000100

{'ts':json_message['lastUpdateId'],
  'bp1':json_message['bids'][0][0],
  'bp1':json_message['bids'][0][0],
  'bp1':json_message['bids'][0][0],
  'bp1':json_message['bids'][0][0],
  'bp1':json_message['bids'][0][0],
  'bp1':json_message['bids'][0][0],
  'bp1':json_message['bids'][0][0],
  'bp1':json_message['bids'][0][0],
  'bp1':json_message['bids'][0][0],
  'bp1':json_message['bids'][0][0],  
  'bp1':json_message['bids'][0][0],
  }

symbol,ts,last_update_id,bp1,bs1,bp2,bs2,bp3,bs3,bp4,bs4,bp5,bs5,bp6,bs6,bp7,bs7,bp8,bs8,bp9,bs9,bp10,bs10,bp11,bs11,bp12,bs12,bp13,bs13,bp14,bs14,bp15,bs15,bp16,bs16,bp17,bs17,bp18,bs18,bp19,bs19,bp20,bs20,ap1,as1,ap2,as2,ap3,as3,ap4,as4,ap5,as5,ap6,as6,ap7,as7,ap8,as8,ap9,as9,ap10,as10,ap11,as11,ap12,as12,ap13,as13,ap14,as14,ap15,as15,ap16,as16,ap17,as17,ap18,as18,ap19,as19,ap20,as20

In [ ]:
import pyspark
from pyspark.streaming import StreamingContext

In [ ]:
#Uncomment the next line to run the code block on jupyter. Keep it commented if copy-pasting into the pyspark shell
sc = pyspark.SparkContext()

# This tells Spark Streaming to bacth-up the contents of a data stream and "ingest" them every 10 seconds.
ssc = StreamingContext(sc,10)

# Tell spark to listen on port 9999 of our localhost.
lines = ssc.socketTextStream('wss://stream.binance.com/ws/BTCUSDT@depth5', 9443)
lines = ssc.socketTextStream('localhost', 9999)    

words = lines.flatMap(lambda line : line.split(" "))

pairs = words.map(lambda word: (word, 1))
wordCount = pairs.reduceByKey(lambda a, b: a + b)

wordCount.pprint()

ssc.start()
ssc.awaitTermination()

In [ ]:
#Uncomment the next line to run the code block on jupyter. Keep it commented if copy-pasting into the pyspark shell
sc = pyspark.SparkContext()

# This tells Spark Streaming to bacth-up the contents of a data stream and "ingest" them every 10 seconds.
ssc = StreamingContext(sc,10)

# Tell spark to listen on port 9999 of our localhost.
lines = ssc.socketTextStream('wss://stream.binance.com/ws/BTCUSDT@depth5', 9443)



lines.pprint()

ssc.start()
ssc.awaitTermination()

In [ ]:
ssc.stop()